# Struttura dei test

Per la fase di test sono stati generati dei dataset tramite la funzione
`make_classification` del modulo `scikit-learn`. Ogni dataset ha un totale di
10000 punti, 990 dei quali sono stati utilizzati per il training dei modelli.
Segue che l'algoritmo genetico è stato eseguito su un totale di 10 punti per
dataset. I parametri che variano da un dataset all'altro sono:

- **Numero di feature di ogni punto**: 2, 4, 8, 16, 32 e 64.
- **Numero di classi possibili per ogni punto**: 2.
- **Seme di casualità per il generatore**: 0, 1.

Per i test effettuati sono stati presi in considerazione tre modelli differenti
del modulo `scikit-learn`: `RandomForestClassifier`, `SVC` e `MLPClassifier`.
Su ognuno dei dataset è stato eseguito l'algoritmo genetico usando tutti e tre
i modelli e, per ognuno di essi, popolazioni sintetiche da 2000, 8000 e 16000
individui.

## Dataset dei risultati

Ogni riga del dataset contiene:

- **Informazioni sul dataset**:
  - **samples**: numero di punti, per ciascun dataset, su cui è stato eseguito
    l'algoritmo genetico.
  - **features**: numero di feature dei punti nel dataset.
  - **classes**: numero di classi possibili per un determinato dataset.
  - **clusters**: numero di cluster per classe.
  - **seed**: seme di casualità con cui è stato generato il dataset.
- **Parametri algoritmo genetico**:
  - **population_size**: numero di individui sintetici generati.
  - **point**: ID del punto del dataset da cui vengono generati i dati
    sintetici.
  - **class**: classe del punto.
  - **target**: classe target dell'algoritmo genetico.
  - **model**: il modello utilizzato per la classificazione.
- **Risultati**:
  - **min/mean/max_fitness**: valori minimo, medio e massimo di fitness, della
    popolazione generata dall'algoritmo genetico.
  - **fitness_std**: deviazione standard dei valori di fitness della popolazione
    sintetica finale.
  - **accuracy**: valore di _precisione_ dell'algoritmo genetico. Si tratta
    della percentuale di individui sintetici classificati nella classe target.

Possiamo quindi vedere ogni riga come una singola esecuzione dell'algoritmo
genetico su uno specifico punto, di uno specifico dataset e con una specifica
classe target.

Procediamo analizzando i risultati mettendo il focus su un paio di parametri
alla volta. Per ogni classificatore verrano svolte analisi separate.

Iniziamo con il valutare i risultati ottenuti dai diversi modelli con un
diverso numero di individui sintetici.

## Conversione fitness-distanza

Dato che i valori di fitness non sono altro che la distanza di ogni punto
sintetico dal punto preso in esame, moltiplicata per un peso di $-1$, possiamo
convertire le tre colonne di fitness in valori di distanza rimoltiplicandole
per $-1$ di modo da avere valori meglio interpretabili.

Ovviamente ciò che era considerato un valori di fitness minimo, diventerà
un valore di distanza massimo (stesso discorso per i valori di fitness massimi).


In [26]:
import pandas as pd

rf = pd.read_csv("../results/ppga_RandomForestClassifier.csv")
svm = pd.read_csv("../results/ppga_SVC.csv")
mlp = pd.read_csv("../results/ppga_MLPClassifier.csv")
ppga_df = pd.concat([rf, svm[svm["clusters"] == 1], mlp[mlp["clusters"] == 1]], axis=0)
ppga_df.sort_values(
    by=["model", "features", "seed", "population_size", "point", "class", "target"],
    inplace=True,
)

ppga_df[["min_fitness", "mean_fitness", "max_fitness"]] *= -1.0
ppga_df = ppga_df.rename(
    columns={
        "min_fitness": "max_distance",
        "mean_fitness": "mean_distance",
        "fitness_std": "distance_std",
        "max_fitness": "min_distance",
    }
).reset_index(drop=True)

ppga_df

,samples,features,classes,clusters,seed,population_size,point,class,target,model,max_distance,mean_distance,distance_std,min_distance,accuracy
0,10,2,2,1,0,2000,0,1,0,MLPClassifier,0.648398,0.643228,0.002557,0.640556,1.0
1,10,2,2,1,0,2000,0,1,1,MLPClassifier,0.021060,0.011172,0.005901,0.000177,1.0
2,10,2,2,1,0,2000,1,0,0,MLPClassifier,0.018436,0.009477,0.005808,0.000048,1.0
3,10,2,2,1,0,2000,1,0,1,MLPClassifier,2.306610,2.297851,0.005649,2.286893,1.0
4,10,2,2,1,0,2000,2,0,0,MLPClassifier,0.021327,0.010399,0.006173,0.000052,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1775,10,64,2,1,1,16000,7,0,1,SVC,18.034405,17.892124,0.087346,17.630542,1.0
1776,10,64,2,1,1,16000,8,0,0,SVC,0.600466,0.382312,0.148695,0.005404,1.0
1777,10,64,2,1,1,16000,8,0,1,SVC,17.296355,16.725999,0.566120,15.312778,1.0
1778,10,64,2,1,1,16000,9,1,0,SVC,15.960198,15.937814,0.018706,15.844560,1.0


In [27]:
import matplotlib.pyplot as plt


def plot_dist(df: pd.DataFrame):
    models = df["model"].unique()
    sizes = df["population_size"].unique()

    fig, axes = plt.subplots(len(models), len(sizes), figsize=(16, 16))

    for i, model in enumerate(models):
        for j, size in enumerate(sizes):
            same = (
                (df["model"] == model)
                & (df["class"] == df["target"])
                & (df["population_size"] == size)
            )
            diff = (
                (df["model"] == model)
                & (df["class"] != df["target"])
                & (df["population_size"] == size)
            )
            axes[i][j].set_title(f"{model} distance distribution\n{size} individuals")
            axes[i][j].set_xlabel("Distance")
            axes[i][j].set_ylabel("Frequency")

            axes[i][j].hist(
                df[same]["mean_distance"],
                edgecolor="black",
                alpha=0.8,
                label="Same class",
            )
            axes[i][j].hist(
                df[diff]["mean_distance"],
                edgecolor="black",
                alpha=0.5,
                label="Different class",
            )
            axes[i][j].legend()

    plt.tight_layout()
    plt.show()


# plot_dist(ppga_df)

## DEAP


In [28]:
rf = pd.read_csv("../results/deap_RandomForestClassifier.csv")
svm = pd.read_csv("../results/deap_SVC.csv")
mlp = pd.read_csv("../results/deap_MLPClassifier.csv")
deap_df = pd.concat([rf, svm[svm["clusters"] == 1], mlp[mlp["clusters"] == 1]], axis=0)
deap_df.sort_values(
    by=["model", "features", "seed", "population_size", "point", "class", "target"],
    inplace=True,
)

deap_df[["min_fitness", "mean_fitness", "max_fitness"]] *= -1.0
deap_df = deap_df.rename(
    columns={
        "min_fitness": "max_distance",
        "mean_fitness": "mean_distance",
        "fitness_std": "distance_std",
        "max_fitness": "min_distance",
    }
).reset_index(drop=True)

deap_df

,samples,features,classes,clusters,seed,population_size,point,class,target,model,max_distance,mean_distance,distance_std,min_distance,accuracy
0,10,2,2,1,0,2000,0,1,0,MLPClassifier,0.616464,0.614860,0.000726,0.614037,1.0
1,10,2,2,1,0,2000,0,1,1,MLPClassifier,0.012599,0.006503,0.003308,0.000511,1.0
2,10,2,2,1,0,2000,1,0,0,MLPClassifier,0.020069,0.009719,0.005738,0.000026,1.0
3,10,2,2,1,0,2000,1,0,1,MLPClassifier,2.312854,2.307666,0.003092,2.302653,1.0
4,10,2,2,1,0,2000,2,0,0,MLPClassifier,0.014586,0.007093,0.004376,0.000221,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,10,64,2,1,1,16000,7,0,1,SVC,19.955289,19.671390,0.231801,18.908130,1.0
2156,10,64,2,1,1,16000,8,0,0,SVC,1.689840,0.937734,0.455527,0.014578,1.0
2157,10,64,2,1,1,16000,8,0,1,SVC,11.423356,11.265149,0.114972,10.905789,1.0
2158,10,64,2,1,1,16000,9,1,0,SVC,15.580845,15.461501,0.097153,15.115339,1.0


In [29]:
# plot_dist(deap_df)

In [31]:
ppga_df = ppga_df[ppga_df["model"] == "MLPClassifier"]
deap_df = deap_df[deap_df["model"] == "MLPClassifier"]
diff = ppga_df["mean_distance"] - deap_df["mean_distance"]

print(f"PPGA better than DEAP: {len(diff[diff < 0]) / len(diff) * 100.0:.2f}%")
print(f"DEAP better than PPGA: {len(diff[diff > 0]) / len(diff) * 100.0:.2f}%")

PPGA better than DEAP: 65.28%
DEAP better than PPGA: 34.72%


## Parallelismo e qualità delle soluzioni

Prima di concludere vorrei evidenziare alcune possibili criticità quando si
lavora con la versione parallela di PPGA, prendendo sempre come riferimento il
solito problema. A differenza di DEAP, che parallelizza solo la fase di
valutazione, PPGA parallelizza anche le fasi di crossover, mutazione.

Durante la fase di test sono infatti emerse alcune criticità riguardanti tali
operatori, che in contesto di calcolo parallelo potrebbero portare a
comportamenti inattesi. Le considerazioni di seguito non riguardano le
performance in termini di runtime ma in termini di qualità delle soluzioni.

### Crossover

Dato che la genereazione della popolazione iniziale avviene per copia, la
diversità genetica iniziale è pari a zero. Per un algoritmo genetico, questo
è in generale un problema dato che sfrutta meccanismi come il crossover per
ricombinare i cromosomi tra individui diversi.

Nel nostro caso partiamo però da una situazione in cui tutti gli individui sono
cloni e utilizziamo metodi di crossover come quello a singolo o doppio punto di
crossover. Questi metodi non producono nuovi valori per le feature ma si
limitano ad esplorare i valori già esistenti.

Ne segue che l'applicazione del crossover tra due cloni risulterà nella
generazione di altri due cloni identici tra di loro e ai loro genitori.

### Mutazione

Per le prime iterazioni ci si affida quindi all'operatore di mutazione come
unico meccanismo per introdurre diversità genetica. Il problema è che la
mutazione ha generalmente una probabilità bassa di essere applicata (in genere
20/30%). Se aggiungiamo poi che ogni feature ha una probabilità indipendente di
essere modificata, la probabilità che un individuo scelto per la mutazione,
venga effettivamente modificato, cala ulteriormente.

Quando partizioniamo la popolazione tra molti worker si potrebbe andare
incontro ad un problema nel caso in cui il rapporto

$$\frac{N}{W}$$

dove $N$ è il numero di individui e $W$ è il numero di worker, è troppo basso.
Si rischia che i worker mutino pochi individui o nessuno, lasciando la
diversità genetica bassa per diverse iterazioni.

Per evitare che ciò accada c'è bisogno che la probabilità indipendente di
modificare ogni feature faccia sì che mediamente, almeno una feature venga
modificata. Per esempio ponendola a $1 / L$ dove $L$ è la lunghezza del
cromosoma.

In secondo luogo è necessario che il numero di individui inviato ad ogni worker
sia abbastanza grande da "garantire" un certo numero di individui mutati.

### Seme di casualità

Dato che le fasi di crossover e mutazione fanno leva sulla generazione di
numeri casuali, è necessario avere fare alcune considerazioni. Il **seme** di
casualità deve essere inizializzato per ogni worker in modo da essere sicuri
che sia diverso. Se non inizializzato esplicitamente verrà usata la funzione
`time.time()`, la quale ha una risoluzione dell'ordine dei secondi.

Ne seguirebbe che ogni worker è inizializzato quasi sicuramente con lo stesso
seed e quindi muta gli stessi individui allo stesso modo degli altri worker.
Andiamo quindi a creare $W$ sottopopolazioni tutte identiche tra di loro.
